# Course 1: Lidar
## Part 2: Point Cloud Segmentation 
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Sensor Fusion Nanodegree programme offered at Udacity.

## Objectives

* Use the [Point Cloud Library](https://pointclouds.org/) (PCL) to segment point clouds;
* Implement the [Random Sample Consensus](https://en.wikipedia.org/wiki/Random_sample_consensus) (RANSAC) algorithm for planar model fitting;
* Perform ground plane separation to "remove" the road surface from our point cloud data.

## 1. Introduction

### 1.1. Point Cloud Segmentation

Our goal in this lesson is to _process_ the point cloud data we received from the LiDAR sensor and _segment_ it in such a way that we are able to "remove" any of the points from our scene which do not belong to the obstacles we wish to detect. In the driving domain, one common type of points in which we wish to remove are _ground plane_ points. These are the points which "belong" to the road surface(s) that the vehicle(s) travel along. 

There are many potential use cases for LiDAR sensors in the automotive space, with point cloud segmentation being one such use case discussed in detail here. There are many other uses for LiDAR sensors and point clouds in the automotive and robotics space: multi-modal sensor calibration / fusion techniques, semi-supervised deep learning models for depth estimation, and for bench-marking tasks used to e.g., evaluate the performance of a "stand-alone" sensor on a given task (such as RGB camera-based vision systems on the monocular depth estimation task).

In the following section, we will narrow our scope to focus only on ground plane estimation as it relates to point cloud segmentation in the section below...

#### Ground Plane Estimation: Problem Setup

In this subsection we will review the topic of ground plane estimation, which is a preliminary exercise for point cloud segmentation and is essential to performing obstacle detection on LiDAR point cloud data. Ground plane estimation involves "detecting" and segmenting points in a point cloud which belong to the ground surface(s) in a scene. Using a technique known as planar fitting, we can extract the points from a levelled surface and remove them from the point cloud. Once these points are effectively discarded, we can begin to segment the object(s) or obstacle(s) for upstream tasks such as detection, tracking, classification, etc.

One primary assumption we will make about the surface we wish to segment is that it is level. In other words, we assume the ground plane / road surface our ego-vehicle is travelling on is _non-inclined_, i.e., that it has a $0^{\circ}$ elevation coinciding with the normal vector (usually defined as the vector perpendicular to the road surface; the upward-facing $z$-axis). To further simplify this problem, we also assume that the surface the ego-vehicle is travelling on is a uniform [planar surface](https://en.wikipedia.org/wiki/Planar_Riemann_surface). In other words, we assume there are no imperfections or deviations along the surface, which would cause the surface to deviate from a smooth, continuous composition. This implies that a constant elevation and constant slope exist along the surface's length and width. In the real-world, road surfaces have elevation, slope, and [textures](https://en.wikipedia.org/wiki/Road_texture) — design elements which break the simplifications we are forming for this toy problem. While real-world road surfaces complicate our rudimentary approach to road plane segmentation, advancements in R&D have been able to handle the non-uniform geometries that exist with non-planar road surfaces by employing various techniques such as deep learning feature extraction [1].   

In our toy example, we proceed with a planar surface assumption that will allow us to form a set of geometric constraints onto our environment which will help us better project a "surface" used to blanket the potential LiDAR points we wish to include in the segmented point cloud. Such a simplification will allow us to reduce the plane fitting estimation into an exercise of "best-fit". Using the [RANSAC](https://en.wikipedia.org/wiki/Random_sample_consensus) algorithm, we can "work out" which points belong to the ground plane / simplified road surface by randomly-sampling a small set of points, fitting a "plane equation" to those sampled points, then performing an iterative two-step inlier "verification" and "consensus" stage to refine the planar surface estimation. It is with this iterative estimation algorithm we can determine a ground plane equation which "fits" or includes the most inlier (true positive) points, leaving us with a "segmentation" of sorts that includes most, if not all, the points belonging to our modelled road surface.

#### Further Considerations for Autonomous Vehicles

We discussed above the necessary trade-offs of our ground plane segmentation problem as it relates to real-world road surfaces. Now, we briefly address the additional constraints placed upon the solution space that come with applications designated for autonomous vehicles. One of those constraints is _robustness_; the ability of an algorithm to appropriately segment a multitude of road surfaces in a variety of conditions is crucial. One grid-based technique, known as multi-level segmentation, handles the diversity in road surfaces by breaking apart the point cloud into smaller chunks to analyse the subsections separately (a "divide-and-conquer" strategy). Another way to account for dynamic road surface characteristics is to use statistical methods to define local features or adjacent points (e.g., Iterative Closest Point). These approaches work by defining a set of parameters (e.g., distance thresholds, point densities, terrain characteristics) to better handle (or adapt) to these challenges. 

With any approach, a very real constraint must be considered. That is, the need for real-time processing. Efficient algorithms are critical, as autonomous vehicles must quickly and effectively perform segmentation on incoming LiDAR data. Therefore, scalable solutions are usually at the forefront of a researcher's mind when it comes to designing for autonomous vehicles.

#### Techniques for Ground Plane Estimation

As organised by T. Gomes [2] in the discussion presented (and lightly summarised by GPT) below, there are five main categories of ground plane estimation techniques that have been explored for the autonomous vehicle use case.

1. **Grid-based Segmentation**: this technique involves breaking up the road surface into 2.5D grid-space known as "tessellations". This grid-based representation of 3D space can reduce computational complexity and alleviate memory demands;
   * Elevation maps — each discretised cell in a "map" is assigned a value based on all points which surround it (e.g., mean average height); the elevation map technique relies on a surface gradient calculation, a clustering of adjacent points, and a "correction" step with respect to all points is calculated. Rudimentary elevation map-based approaches may fall short of real-time requirements and have shown limited ability to simultaneously model multiple surfaces somewhat frequently seen in the driving domain (e.g., road surface and the "terrain above a [freeway] underpass");
   * Occupancy grid maps — regarded as "extremely robust and easy to implement" [3], occupancy grid-based techniques exploit a trade-off between resolution and computational complexity (exploitation versus exploration). The occupancy grid approach helps handle the ambiguities that exists in noisy sensor data, accommodating for the various reflective properties (absorption and dispersion) of the surface being modelled;
2. **Ground Modelling with Object Maps**: this technique enforces geometric constraints and simplifications on the environment which allow the ground surface to be modelled by a set of geometric shapes or objects, such a single plane or a grid of planar surfaces. Our approach of planar fitting falls under this category;
   * Planar fitting — assumes that the ground surface can be modelled as a large, flat, uniform plane. Using estimation algorithms such as RANSAC allows for the efficient estimation of the plane's unknown parameters. However, it should be obvious that most road surfaces cannot be simply modelled as flat surfaces. Therefore, despite the computational efficiency, planar fitting with the RANSAC algorithm is not a robust enough solution to appropriately handle the complexities of the real world;
   * Line extraction — works by dividing the scene data into small patches. Within each patch, lines are fitted to capture the dominate linear trends. These lines can help identify edges or boundaries within the ground plane, such as curbs or changes in ground slope. Also apparent in this method its sensitivity to highly-uneven road surfaces, such as with off-road terrain found in the [DARPA Grand Challenge](https://en.wikipedia.org/wiki/DARPA_Grand_Challenge).
3. **Adjacent Points and Local Features**: these techniques explore the spatial relationships between neighbour points and local geometric features;
   * Channel-based — analyses the relationship between points contained in the vertical "channels" or bands of a LiDAR scan. Effectively handles uneven terrain but struggles in complex environments due to its dependence on parameterisation and its ignorance to the inherent horizontal channel-based relationships;
   * Region-growing — exploits a set of "similarity criteria" to fit a "region" around a seed point. While this method is easy to implement, results are highly dependent on choosing a good starting seed. Complex urban areas are one domain which this technique fails to handle effectively;
   * Clustering — groups point cloud data into segments or "clusters" based on height, or with features such as shape, or geometries such as surface normals. This voxelisation technique may be robust, it struggles in areas with low point-density and is computationally expensive to run over datasets with multi-dimensional features;
   * Range images — exploits "neighbourhood" relations (trigonometric relationships between adjacent points) found in 2D range images which have been converted from LiDAR point clouds. Real-time segmentation is possible using an angle-based approach, but accuracy can be dependent on the range image conversion process and through complexities related to the sensor model;
4. Higher-Order Inference: attempts to handle the sparsity in data which exists in LiDAR point clouds at farther distances;
   * Markov Random Field (MRF) — uses a probabilistic graph model to consider relationships between points in an undirected graph structure. Can be effective in handling sparse point clouds, but is computationally expensive and not robust enough to handle very rough terrain. A subset of MRF approaches aim to reduce computational complexity, but these algorithm- or data structure-dependent approaches suffer the same fate with respect to real-time demands;
   * Conditional Random Field (CRF) — captures long-range dependencies between points by labelling sequences of nodes given a chain of observations. Using temporal constraints, the spatio-temporal relationships are analysed to form a ground plane. Multiple iterations of CRF are required for uneven terrains, which comes at the cost of any real-time performance outlook.
5. Deep learning-based methods: a set of techniques using neural networks to perform segmentation, usually at the expense of computational efficiencies;
   * PointNet / PointNet++ — using methods formally suited for computer vision-related tasks, PointNet models extract local features with a hierarchical neural network designed to recognise fine-grained patterns and generalise to complex scenes. PointNet++ better learns from point clouds with varying point densities, but overall these methods require greater computational complexity in training and at inference time;
   * VoxelNet / PointPillar — discretises point clouds into voxels or vertical columns (pillars), while relying on a modified PointNet for the feature encoder module. Achieves good results, but requires additional computational complexity;
   * SqueezeSeg / RangeNet++ — utilises 2D representations (birds-eye view images), relies on a CRF backbone implemented as a recurrent layer. Achieves faster performance at the cost of accuracy;
   * GndNet — achieves real-time performance (55Hz) by converting the point cloud into a 2D grid, uses a modified PointNet and 2D CNN as the backbone.
   
   
In addition to the above, there are many other deep learning-based techniques which seek to optimise run-time efficiency using hardware accelerators (FPGAs; Lyu et al.) or hardware-dependent optimisation methods. However, the challenges regarding high computational demands and need for large amounts of labelled training data still remain. 

## 2. Programming Task

#### Ground plane estimation with RANSAC

To start, we will perform ground plane segmentation of our LiDAR point cloud data using the estimation technique proposed by Hu et al. [4]. This is a RANSAC-based solution which "fits" a geometrically-modelled ground plane to the largest set of "inliers" — LiDAR points estimated to be belonging to the plane based on a minimum distance threshold.

The RANSAC algorithm works to iteratively determine the "minimum" distance needed to safely assign the inlier points to the ground plane. Starting by assigning the lowest elevation points contained in each frame, the RANSAC algorithm then calculates the error (distance) of the points to the estimated plane to minimise over. Then, the iteration and consensus stage is performed, where the modelled plane containing the most inliers will be kept. On truly flat surfaces, such as the one we will model, RANSAC-based planar fitting methods have been shown to achieve an effectiveness of "around 81%" on the KITTI dataset [5].

#### Using the Point Cloud Library (PCL)

In our first part of the programming task, we use the built-in functions contained in the Point Cloud Library (PCL) to perform ground plane segmentation. In our [`SFND_Lidar_Obstacle_Detection`]() project codebase, we define a `pointProcessor` object which will contain the helper functions that call the built-in PCL functions repsonsible for the core task of segmentation. 

For further clarification, see the Point Cloud Library tutorial page on [planar segmentation with PCL](https://pointclouds.org/documentation/tutorials/planar_segmentation.html#planar-segmentation). Let's get started...

##### E1.2.1: `processPointClouds`

In the [`/src/environment.cpp`]() file, we create an object instance of the `processPointClouds` class. The choice to instantiate this object on the stack or the heap is up to you. In the `initCamera()` function we create the `pointProcessor` objects onto the heap by initialising the object pointers, then setting them to point to the memory addresses created with the `new` keyword. However, if you would like to instantiate the object onto the stack, then observe in `simpleHighway()` the declaration of the new `ProcessPointCloud` class objects.   

Defined in [`/src/processPointClouds.h`](), this object contains all the class functions needed to load / save, filter, extract and segment the point cloud objects using Point Cloud Library.  

```cpp
// In `/src/environment.cpp`:

void initCamera(
    CameraAngle setAngle,
    pcl::visualization::PCLViewer::Ptr& viewer
) {

  // ..
  /** E1.2.1: Create a point processor. **/
  // Here, we instantiate the `ProcessPointClouds` instances on the stack.
  // To instead use the heap, see `simpleHighway()`.
  // Instantiating two instances; one for 3D points,
  // the other for 3D + intensity-valued points.
  //ProcessPointClouds<pcl::PointXYZ>* pointProcessor = new ProcessPointClouds<pcl::PointXYZ>();
  //ProcessPointClouds<pcl::PointXYZI>* pointProcessor = new ProcessPointClouds<pcl::PointXYZI>();
}


void simpleHighway(
    pcl::visualization::PCLVisualizer::Ptr& viewer
) {
    // ..
    /** E1.2.1: Create a point processor. **/
    // Here, we instantiate the `ProcessPointClouds` instances on the heap.
    // To instead use the stack, see `initCamera()`.
    // TODO:: Create point processor
    ProcessPointClouds<pcl::PointXYZ> pointProcessorXYZ;
    ProcessPointClouds<pcl::PointXYZI> pointProcessorXYZI;
    // ..
```

##### E1.2.2: `SegmentPlane`

Here we define the `SegmentPlane` function found in the [`/src/processPointClouds.h`]() file. Note that in this function we make use of the `typename`-decorated `PointT` template parameter. This allows us to reference "generic" point types without needing to make specific instances of this function to handle individually `PointXYZ` or `PointXYZI` data types (as done previously in E1.2.1).

```cpp
// In `/src/processPointClouds.cpp`:

template<typename PointT> std::pair<
    typename pcl::PointCloud<PointT>::Ptr, 
    typename pcl::PointCloud<PointT>::Ptr
> ProcessPointClouds<PointT>::SegmentPlane(
    typename pcl::PointCloud<PointT>::Ptr cloud, 
    int maxIterations, 
    float distanceThreshold
) {
  // ..
  /** E1.2.2: Perform plane segmentation with PCL. **/
  // TODO:: Fill in the function to segment cloud into two parts, the drivable plane and obstacles
  // Creating a new PCL segmentation class instance
  pcl::SACSegmentation<PointT> seg;
  // Creating intermediary objects consumed by the PCL algorithm
  pcl::PointIndices::Ptr inliers{new pcl::PointIndices};
  pcl::ModelCoefficients::Ptr coefficients{new pcl::ModelCoefficients};
  // Configuring the segmentation parameters
  seg.setOptimizeCoefficients(true);
  seg.setModelType(pcl::SACMODEL_PLANE);
  seg.setMethodType(pcl::SAC_RANSAC);
  seg.setMaxIterations(maxIterations);
  seg.setDistanceThreshold(distanceThreshold);
  // Segmenting the largest planar component from the input cloud
  seg.setInputCloud(cloud);
  seg.segment(*inliers, *coefficients);
  if (inliers->indices.size() == 0) {
    std::cout << "Could not estimate a planar model for the given dataset.\n";
  }
  // ..
}

```

##### E1.2.3: `SeparateClouds`

Now that we have obtained the inlier points belonging to the ground plane, we want to "split" the entire point cloud into two: one containing the ground plane, and the other containing all other points. To do this, we will fill out the `SeparateClouds()` function inside the [`/src/processPointClouds.cpp`]() file.

Using the `extract()` function provided to use in the Point Cloud Library, we can remove the inliers  in one point cloud from the other point cloud. To set this up, we must first take the set of inliers and copy them to a new PCD: the `ground`. For the other points, we create a `obstacles` instance. 

```cpp
// In `/src/processPointClouds.cpp`:

template<typename PointT> std::pair<
    typename pcl::PointCloud<PointT>::Ptr, 
    typename pcl::PointCloud<PointT>::Ptr
> ProcessPointClouds<PointT>::SeparateClouds(
    pcl::PointIndices::Ptr inliers, 
    typename pcl::PointCloud<PointT>::Ptr cloud
) {
    /** E1.2.3: Separating the ground plane. **/
    // TODO: Create two new point clouds, one cloud with obstacles and other with segmented plane
    typename pcl::PointCloud<PointT>::Ptr obstacles(new pcl::PointCloud<PointT>());
    typename pcl::PointCloud<PointT>::Ptr ground(new pcl::PointCloud<PointT>());
    // Creating the filtering object
    pcl::ExtractIndices<PointT> extract;
    // Extracting the given `inliers` from the `cloud`
    extract.setInputCloud(cloud);
    extract.setIndices(inliers);
    extract.setNegative(false);
    // Storing extracted indices in output cloud
    extract.filter(*ground);
    std::cerr << "PointCloud representing the planar component: " 
              << ground->width * ground->height << " data points.\n";
    // Fetching the remaining non-extracted indices
    // outliers = extract.getRemovedIndices();
    // Copying over the remaining points into `obstacles` cloud
    extract.setNegative(true);
    extract.filter(*obstacles);
    std::cerr << "PointCloud representing the obstacles component: " 
              << obstacles->width * obstacles->height 
              << " data points.\n"; 
    std::pair<
        typename pcl::PointCloud<PointT>::Ptr, 
        typename pcl::PointCloud<PointT>::Ptr
    > segResult(ground, obstacles);
    return segResult;
}

```

##### E1.2.4: Rendering the segmentation results

Below we will call the functions we defined above to segment the point cloud scan into two separate obstacles and the ground plane instances. Then, we will render the point clouds onto the viewer and observe the results.

```cpp
// In `/src/environment.cpp`:

void simpleHighway(
    pcl::visualization::PCLVisualizer::Ptr& viewer
) {
    // ..
    /** E1.2.1: Create a point processor. **/
    // Here, we instantiate the `ProcessPointClouds` instances on the stack.
    // However, we can instantiate instead on the heap (see `initCamera()`).
    // TODO:: Create point processor (type-dependent)
    ProcessPointClouds<pcl::PointXYZ> pointProcessor;
    // ProcessPointClouds<pcl::PointXYZI> pointProcessorXYZI;
    /** E1.2.3: Separating the ground plane. **/
    // Fetching the resulting point clouds
    // Here, we access the `pointProcessor` class function using the
    // "dot" accessor, since the object was instantiated on the stack. 
    std::pair<
      pcl::PointCloud<pcl::PointXYZ>::Ptr,
      pcl::PointCloud<pcl::PointXYZ>::Ptr
    > segmentCloud = pointProcessor.SegmentPlane(
        pointCloud,
        100,
        0.2
    );
//     std::pair<
//         typename pcl::PointCloud<PointT>::Ptr,
//         typename pcl::PointCloud<PointT>::Ptr
//     > segmentCloud = pointProcessorT.SegmentPlane(
//         pointCloud,
//         100,
//         0.2
//     );
    /** E1.2.4: Rendering the clouds onto the viewer **/
    renderPointCloud(
	viewer,
        segmentCloud.first,
        "ground",
        Color(0, 1, 0)
    );
    renderPointCloud(
        viewer,
	segmentCloud.second,
        "obstacles",
        Color(1, 0, 0)
    );
}
```

<img src="figures/2024-04-24-Figure-1-Point-Cloud-Segmentation.png" alt="Figure 1. Point Cloud Segmentation: Ground plane (green) and obstacles (red)." height="70%" width="70%">

$$\textrm{Figure 1. Point Cloud Segmentation: Ground plane (green) and obstacles (red).}$$

This concludes Exercises 1.2.1-4. As shown in Figure 1 above, we obtained two distinct point clouds from the input LiDAR sensor scan; a set of `933` points belonging to the estimated ground plane (rendered in green above), and a set of `43` points belonging to the reflected obstacle detections (rendered in red above).

In the following section, we will repeat the activity of ground plane segmentation, but instead implementing the RANSAC algorithm by hand.

#### Implementing RANSAC by hand

##### RANSAC for 2D line fitting

In this section we will be implementing the RANSAC algorithm by hand. First, we simplify the "model" to be a simple line onto which we will fit the set of inlier points to. Rather than fit our model on the data points we previously derived, we will instead sample a set of 2D points (such that the $z$-axis coordinate values are `0`). This simplification allows us to form a 2D line equation and easily calculate the _distances_ between each 2D point and the line of "best fit".

To start, let's cover the basic equations we will be working with. For two variables, $x$ and $y$, and a set of coefficients $\left[\mathrm{A}, \mathrm{B}, \mathrm{C}\right]$, we form the following [linear equation](https://en.wikipedia.org/wiki/Linear_equation):
$$\mathrm{A}x + \mathrm{B}y + \mathrm{C} = 0.$$

These coefficients $\left[\mathrm{A}, \mathrm{B}, \mathrm{C}\right]$ are considered to be _parameters_ of our model, and will be iteratively calculated to best estimate their values which "bend" or shape the line in such a way that it minimises the overall "error". We will discuss this error in just a bit, but let's now fit the equation of a line above to a given point pair, $\left(x_{1}, y_{1}\right)$ and $\left(x_{2}, y_{2}\right)$. This yields the equation:
$$\left(y_{1} - y_{2}\right)x + \left(x_{2} - x_{1}\right)y + \left(x_{1}*y_{2} - y_{1}*x_{2}\right) = 0.$$

To derive the above equation, fit the point-slope form of a line:
$$y - y_{1} = m\left(x - x_{1}\right),$$

for a given point $\left(x_{1}, y_{1}\right)$ on the line. Knowing that the slope $m$ of this line connecting the two points $\left(x_{1}, y_{1}\right)$ and $\left(x_{2}, y_{2}\right)$ can be calculated as:
$$m = \frac{y_{2} - y_{1}}{x_{2} - x_{1}},$$

we substitute the expression for the slope into the point-slope equation. After manipulating the algebraic expression (i.e., distributing the slope term into the parantheses, then re-arranging the expression to isolate the equation with respect to the variable $y$), we obtain:
$$y = \frac{y_{2} - y_{1}}{x_{2} - x_{1}}x - \frac{y_{2} - y{1}}{x_{2} - x_{1}}x_{1} + y_{1}.$$

We can then simplify the expression above by eliminating the denominator $\left(x_{2} - x_{1}\right)$. To do so, multiply all terms by the denominator to obtain:
$$y\left(x_{2} - x_{1}\right) - x\left(y_{2} - y_{1}\right) + x_{1}\left(y_{2} - y_{1}\right) - y_{1}\left(x_{2} - x_{1}\right) = 0.$$

Now, going back to the original equation of a line, $\mathrm{A}x + \mathrm{B}y + \mathrm{C} = 0$, we match the coefficients to the terms derived above. This gives us the following:
$$
\begin{align*}
\mathrm{A} &= -\left(y_{2} - y_{1}\right) = y_{1} - y_{2}, \\
\mathrm{B} &= x_{2} - x_{1}, \\
\mathrm{C} &= x_{1}\left(y_{2} - y_{1}\right) - y_{1}\left(x_{2} - x_{1}\right) \\
&= x_{1}y_{2} - y_{1}x_{1} - y_{1}x_{2} + y_{1}x_{1} \\
&= x_{1}y_{2} - y_{1}x_{2}.
\end{align*}
$$


The above equation of the line connecting the two points, $\left(x_{1}, y_{1}\right)$ and $\left(x_{2}, y_{2}\right)$, will be used as our "model". To determine if this "model" is best suited to represent our data, we will need to iterate over every remaining point in our dataset and calculate the _distance_ from each of those points to the line we just created. This distance between each point and the line will be used to calculate the "error" term, and a threshold value will be used to determine whether or not each point is considered to be an inlier based on the distance computed. To calculate distance $d_{i}$ between the line and a point $\left(x_{i}, y_{i}\right)$, we use the formula:
$$d_{i} = \frac{\vert\mathrm{A}x_{i} + \mathrm{B}y_{i} + \mathrm{C}\vert}{\sqrt{\mathrm{A}^{2} + \mathrm{B}^{2}}}.$$


 The distance $d_{i}$ between the line and the point is checked against the pre-determined threshold value `distanceTol`, and, if $d_{i}$ is determined to be lesser than the threshold, the point $\left(x_{i}, y_{i}\right)$ is labelled as an "inlier" point. Otherwise, the point is labelled as an "outlier". The RANSAC algorithm we are implementing here will iterate over the remaining points left in the dataset, then count how many points were added to the inlier set. After this calculation, an entirely new line equation is fit (with two other points chosen at random), and the distance calculation and inlier counting process is repeated again. After a pre-determined number of model iterations `maxIterations` is performed, the "model" with highest number of inliers is returned (i.e., the two points which make up the line equation).

##### E1.2.5: `Ransac`

```cpp
// In `src/quiz/ransac/ransac2d.cpp`:

std::unordered_set<int> Ransac(
		pcl::PointCloud<pcl::PointXYZ>::Ptr cloud, 
		int maxIterations, 
		float distanceTol
) {
	// Storing the inliers of the "best fit" model
	std::unordered_set<int> inliersResult;
  // Initialising the random number generator
	srand(time(NULL));
  /*** E1.2.5: Perform RANSAC for 2D line fitting. ***/
	// TODO: Fill in this function
	// Storing greatest number of inliers found
	int bestNumInliersFound = std::numeric_limits<int>::min();
	/** Performing model fitting for max iterations **/
	for (int i = 0; i < maxIterations; i++) {
		// Randomly sample subset and fit line
		// Storing the inliers of this current line
		std::unordered_set<int> inliersTemp;
		/** Sampling two points "at random" **/
		// Getting number of points total in point cloud
		int numPoints = (int)cloud->size();
		// Selecting two point indices "at random" 
		int pointIdx1 = rand() % numPoints;
		int pointIdx2 = rand() % numPoints;
		// Fetching the two randomly-selected points
		pcl::PointXYZ p1 = cloud->points[pointIdx1];
		pcl::PointXYZ p2 = cloud->points[pointIdx2];
		/** "Fitting" the equation of a line to the two points **/
		double A = p1.y - p2.y;
		double B = p2.x - p1.x;
		double C = (p1.x * p2.y) - (p1.y * p2.x);
		/** Computing point-line distance over all points **/
		// Counting number of current inlier points with distances less than threshold
		int numInliersCurrent = 0;
		for (int j = 0; j < numPoints; j++) {
			std::cerr << "Iteration " << j << ": "
								<< "numInliersCurrent = " << numInliersCurrent
								<< ", p1 = " << p1
								<< ", p2 = " << p2;
			// Fetching point "at random"
			int pointIdxj = rand() % numPoints;
			// Checking if one of anchor points,
			// i.e., point used to fit line
			if ((pointIdxj == pointIdx1)
				  || (pointIdxj == pointIdx2)
			) {
				// Randomly-selected point is an anchor point,
				std::cerr << "\nAnchor point encountered.\n";
				// Add anchor point to set of inliers
				inliersTemp.insert(j);
				// Skip distance calculation to avoid divide-by-zero errors
				continue;
			}
			pcl::PointXYZ p_j = cloud->points[pointIdxj];
			std::cerr << ", p_j = " << p_j;
			// Calculating distance from point $j$ to line $i$
			double d_ji = std::fabs(
				A * p_j.x + B * p_j.y + C
			) / std::sqrt(
				std::pow(A, 2) + std::pow(B, 2)
			);
			std::cerr << ", d_ji = " << d_ji << ".\n";
			// Checking distance against threshold
			if (d_ji <= distanceTol) {
				// Distance is within threshold,
				// Point is considered to be an "inlier"
				numInliersCurrent += 1;
				// Store point index in the current inliers set
				inliersTemp.insert(pointIdxj);
			}
		}
		// Checking if this line fit the "best" (most) number of points
		if (numInliersCurrent >= bestNumInliersFound) {
			// Update the "best" inlier set to be this current one
			inliersResult = inliersTemp;
			bestNumInliersFound = numInliersCurrent;
		}
		// Otherwise, clear this model's inlier set and repeat with new line
		inliersTemp.clear();
		// Reset number of inliers found for the next model iteration
		numInliersCurrent = 0;
	}  // Repeat next model iteration
	// Checking if we obtained a "valid" result
	if (bestNumInliersFound <= 0) {
		// No inliers found; or, error has occurred
		std::cerr << "Error has occurred; no inliers found ("
							<< "bestNumInliersFound: " << bestNumInliersFound 
							<< ").\n";
	}
	// Return indices of inliers from "best" fitted line,
	// i.e., the line that "fit" the most number of inliers.
	return inliersResult;
}
```

##### E1.2.6: `Ransac` model parameters

Now, all that is left to do before we run the `Ransac` function above is to set our experiment run values. The `distance` value and the `maxIterations` are two parameters we can adjust freely to alter the model results.

```cpp

// In `src/quiz/ransac/ransac2d.cpp`:

int main()
{
  /*** E1.2.6: Modify the RANSAC model parameters. ***/
	// TODO: Change the max iteration and distance tolerance arguments for Ransac function
  int maxIterations = 10;
  double distanceTol = 0.5;
  /*** E1.2.5: Perform RANSAC for 2D line fitting. ***/
	std::unordered_set<int> inliers = Ransac(
    cloud, 
    maxIterations, 
    distanceTol
  );
  // ..
}
```

Finally, putting it all together, we obtain the following results:

<img src="figures/2024-04-24-Figure-2-RANSAC-Line-Fitting-By-Hand.png" alt="Figure 2. RANSAC Line Fitting — Resulting line of best fit and set of inliers (shown in green); `maxIterations` = 10; `distanceTol` = 1.5.">

$$\textrm{Figure 2. RANSAC Line Fitting — Resulting line of best fit and set of inliers (shown in green).}$$

To produce the above results, we used a `maxIterations` of `10` and a `distanceTol` of `1.5`.

##### Extending RANSAC to 3D (Plane Fitting)

We will extend our work from E1.2.6 to now fit a planar surface to a set of 3D points. We start by defining the plane with the equation:
$$\mathrm{A}x + \mathrm{B}y + \mathrm{C}z + \mathrm{D} = 0.$$

Just as before, we have a set of constants defining the coefficients of the equation. This time, since we are working in 3D, we have an additional variable $z$ and a trailing constant $\mathrm{D}$.

To fit the plane to our point cloud data, we will choose three _non-collinear_ points:
$$\begin{align} p_{1} &= \left(x_{1}, y_{1}, z_{1}\right), \\ p_{2} &= \left(x_{2}, y_{2}, z_{2}\right), \\ p_{3} &= \left(x_{3}, y_{3}, z_{3}\right). \end{align}$$

With these three points we form two vectors originating from point $p_{1}$; one vector, $v_{1}$, which travels from point $p_{1}$ to point $p_{2}$, and the other vector, $v_{2}$, which travels from point $p_{1}$ to point $p_{3}$. The coordinates of these vectors are calculated by subtracting the coordinates of $p_{1}$ from the coordinates of $p_{2}$ and $p_{3}$, respectively. This gives us:
$$\begin{align} v_{1} &= \left<x_{2} - x_{1}, y_{2} - y_{1}, z_{2} - z_{1}\right>, \\ v_{2} &= \left<x_{3} - x_{1}, y_{3} - y_{1}, z_{3} - z_{1}\right>.\end{align}$$

To form a [normal vector](https://en.wikipedia.org/wiki/Normal_(geometry)) to this plane, we take the cross product of $v_{1} \times v_{2}$, which in expanded form gives us:
$$ v_{1} \times v_{2} =
\left\langle\begin{matrix}
\left(y_{2} - y_{1}\right)\left(z_{3} - z_{1}\right) - \left(z_{2} - z_{1}\right)\left(y_{3} - y_{1}\right) \\
\left(z_{2} - z_{1}\right)\left(x_{3} - x_{1}\right) - \left(x_{2} - x_{1}\right)\left(z_{3} - z_{1}\right) \\
\left(x_{2} - x_{1}\right)\left(y_{3} - y_{1}\right) - \left(y_{2} - y_{1}\right)\left(x_{3} - x_{1}\right)

\end{matrix}\right\rangle.$$

In summary, the cross product of two vectors $v_{1}$ and $v_{2}$ gives us a vector perpendicular to both. To calculate this cross product we take the determinant of a $3\times3$ matrix formed by the components of the vectors $v_{1}$ and $v_{2}$, which is expanded above. To express the result of the cross product in simplified notation, we re-write it in terms of unit vectors $i$, $j$, and $k$, which represent the $x$-, $y$- and $z$-axis directions of the vector.

In simplified notation using unit vectors, this cross product becomes:
$$v_{1} \times v_{2} = \left<i, j, k\right>,$$

which forms our equations:
$$\begin{align} i\left(x - x_{1}\right) + j\left(y - y_{1}\right) + k\left(z - z_{1}\right) &= 0, \\ ix + jy + kz - \left(ix_{1} + jy_{1} + kz_{1}\right) &= 0. \end{align}$$

These two equations are obtained by taking the dot product between the normal vector and a vector formed by a point on the plane. This results in two equivalent forms of the plane equation we derived earlier; one in the form:
$$\mathrm{A}x + \mathrm{B}y + \mathrm{C}z + \mathrm{D} = 0,$$
and:
$$\mathrm{A}x + \mathrm{B}y + \mathrm{C}z = \mathrm{D}.$$

Note that in either of the equivalent forms of the plane equation, the term with the coefficient "$\mathrm{D}$" remains positive. This is to illustrate that a specific _interpretation_ of this constant term is being applied. Given that $\mathrm{D}$ represents the perpendicular distance from the origin to the plane along the direction of the plane's normal vector. This "distance" is represented as a _magnitude_ rather than a signed value, which is irrespective of the direction the distance is travelling along.

Therefore, we can extract the values of the coefficients to obtain:
$$\begin{align} \mathrm{A} &= i, \\ \mathrm{B} &= j, \\ \mathrm{C} &= k, \\ \mathrm{D} &= -\left(ix_{1} + jy_{1} + kz_{1}\right)\end{align}.$$


With the equation of the plane resolved, we can now calculate the distance from the plane to a given point $\left(x_{i}, y_{i}, z_{i}\right)$. Assuming the plane is represented by the following equation:
$$\mathrm{A}x + \mathrm{B}y + \mathrm{C}z + \mathrm{D} = 0,$$

we can calculate the distance $d_{i}$ from this plane to the point $\left(x_{i}, y_{i}, z_{i}\right)$ using:
$$d_{i} = \frac{\vert \mathrm{A} * x + \mathrm{B} * y + \mathrm{C} * z + \mathrm{D}\vert}{\sqrt{\mathrm{A}^{2} + \mathrm{B}^{2} + \mathrm{C}^{2}}}.$$

##### E1.2.7: `RansacPlane`

```cpp
#include <set>

std::unordered_set<int> RansacPlane(
		pcl::PointCloud<pcl::PointXYZ>::Ptr cloud, 
		int maxIterations, 
		float distanceTol
) {
	// Storing the inliers of the "best fit" model
	std::unordered_set<int> inliersResult;
	// Initialising the random number generator
	std::srand(time(NULL));
	/*** E1.2.7: Perform RANSAC for 3D plane fitting. ***/
	int bestNumInliersFound = std::numeric_limits<int>::min();
	/** Performing model fitting for max iterations **/
	for (int i = 0; i < maxIterations; i++) {
		// Storing the inliers of the current plane
		std::unordered_set<int> inliersTemp;
		/** Sampling three points "at random" **/
		// Randomly sample subset of points to fit plane
		int numPoints = (int)cloud->size();
		// Selecting three point indices at "random"
		// TODO: Check for co-linearity with matrix determinant calculation
		// CANDO: Use `pcl::determinant3x3Matrix()` with `Matrix` type;
		// Will need: `#include <pcl/common/eigen.h>`.
		std::set<int> anchorPoints;
		while(anchorPoints.size() < 3) {
			// Making sure the three "anchor" points are not equal,
			// Since the `std::set` will not allow for duplicates.
			anchorPoints.insert(
				rand() % numPoints
			);
		}
		if (anchorPoints.empty()) {
			std::cerr << "Error; cannot form co-linear vectors, "
							  <<  "Must have three unique points.";
			return inliersResult;
		}
		else if (anchorPoints.size() < 3) {
			std::cerr << "Error; not enough unique points found.";
			return inliersResult;
		}
		// Fetching the indices of the three anchor points to use later
		std::set<int>::iterator idx = anchorPoints.begin();
		int pointIdx1 = *idx; idx++;
		int pointIdx2 = *idx; idx++;
		int pointIdx3 = *idx;
		// Fetching the three randomly-selected anchor points for the plane
		pcl::PointXYZ p1 = cloud->points[pointIdx1];
		pcl::PointXYZ p2 = cloud->points[pointIdx2];
		pcl::PointXYZ p3 = cloud->points[pointIdx3];
		/** "Fitting" the equation of a plane to the three points **/
		// First, we form the two vectors originating from `p1`:
		double v1[3] = {
			p2.x - p1.x,
			p2.y - p1.y,
			p2.z - p1.z
		};
		double v2[3] = {
			p3.x - p1.x,
			p3.y - p1.y,
			p3.z - p1.z
		};
		// Secondly, with the cross product we form a normal vector to the plane,
		// Such that $v_{1} \times v_{2} = <i, j, k>$:
		double v1xv2[3] = {
			(p2.y - p1.y) * (p3.z - p1.z) - (p2.z - p1.z) * (p3.y - p1.y),
			(p2.z - p1.z) * (p3.x - p1.x) - (p2.x - p1.x) * (p3.z - p1.z),
			(p2.x - p1.x) * (p3.y - p1.y) - (p2.y - p1.y) * (p3.x - p1.x)
		};
		// Lastly, we extract the values of the coefficients of the plane:
		double A = v1xv2[0];
		double B = v1xv2[1];
		double C = v1xv2[2];
		double D = -(
			v1xv2[0] * p1.x + v1xv2[1] * p1.y + v1xv2[2] * p1.z
		);
		/** Computing point-plane distance over all points **/
		// Counting number of current inliers found
		// "Inlier" here means a point with a distance to the plane
		// less than the given distance threshold value
		int numInliersCurrent = 0;
		for (int j = 0; j < numPoints; j++) {
			std::cerr << "Iteration " << j << ": "
								<< "numInliersCurrent = " << numInliersCurrent
								<< ", p1 = " << p1
								<< ", p2 = " << p2
								<< ", p3 = " << p3;
			// Fetching point "at random"
			int pointIdxj = rand() % numPoints;
			// Checking if one of the anchor points,
			// i.e., the point(s) used to fit the plane
			if ((pointIdxj == pointIdx1)
			  || (pointIdxj == pointIdx2)
				|| (pointIdxj == pointIdx3)  
			) {
				// Randomly-selected point is an anchor point,
				std::cerr << "\nAnchor point encountered.\n";
				// Add anchor point to set of inliers
				inliersTemp.insert(j);
				// Skip distance calculation to avoid divide-by-zero errors
				// CANDO: Case (1) "Coincident" point - point is on plane.
				// CANDO: Case (2) Parallel plane - plane is parallel to coordinate axes.
				// CANDO: Case (3) Numerical instability - very small denominator values.
				// CANDO: Case (4) Infinity distance - plane parallel to point vector.
				continue;
			}
			pcl::PointXYZ p_j = cloud->points[pointIdxj];
			std::cerr << ", p_j = " << p_j;
			// Calculating distance from point $j$ to plane $v_{1} \times v_{2}$
			double d_jv1xv2 = std::fabs(
				A * p_j.x + B * p_j.y + C * p_j.z + D
			) / std::sqrt(
				std::pow(A, 2) + std::pow(B, 2) + std::pow(C, 2)
			);
			// Checking distance against threshold
			if (d_jv1xv2 <= distanceTol) {
				// Distance is within threshold,
				// Point is considered to be an "inlier"
				numInliersCurrent += 1;
				inliersTemp.insert(pointIdxj);
			}
		} // Repeat for all remaining points in point cloud
		// Checking if this plane fit the "best" (most) number of points
		if (numInliersCurrent >= bestNumInliersFound) {
			// Update the "best" inlier set to be this current one
			inliersResult = inliersTemp;
			bestNumInliersFound = numInliersCurrent;
		}
		// Otherwise, clear this model's inlier set and repeat with new plane
		inliersTemp.clear();
		// Reset number of inliers found for the next model iteration
		numInliersCurrent = 0;
	} // Repeat model fitting for maximum number of iterations
	// Checking if we obtained a "valid" result
	if (bestNumInliersFound <= 0) {
		// No inliers found; or, error has occurred
		std::cerr << "Error has occurred; no inliers found ("
							<< "bestNumInliersFound: " << bestNumInliersFound
							<< ").\n";
	}
	// Return indices of inliers from "best" fitted plane,
	// i.e., the plane that "fit" the most number of inliers.
	return inliersResult;
}
```

##### E1.2.8: `RansacPlane` model parameters

Now, all that is left to do before we run the `RansacPlane` function above is to set our experiment run values. The `distance` value and the `maxIterations` are two parameters we can adjust freely to alter the model results.

```cpp

// In `src/quiz/ransac/ransac2d.cpp`:

int main()
{
  /*** E1.2.8: Modify the RANSAC model parameters. ***/
  // First, we create the 3D data needed for this experiment using the helper function
  pcl::PointCloud<pcl::PointXYZ>::Ptr cloud = CreateData3D();
  // Then, we can modify the experiment parameters for this run:
	// TODO: Change the max iteration and distance tolerance arguments for Ransac function
  int maxIterations = 10;
  double distanceTol = 0.5;
  /*** E1.2.7: Perform RANSAC for 3D plane fitting. ***/
  // Calling the RANSAC for 3D plane fitting function:
	std::unordered_set<int> inliers = RansacPlane(
    cloud, 
    maxIterations, 
    distanceTol
  );
  // ..
}
```

## 3. Closing Remarks

#### Alternatives
* Instantiate the `ProcessPointClouds` instance on the heap instead of the stack (or vice-versa);

#### Extensions of task
* Replace all uses of specific `pcl::PointXYZ` or `pcl::PointXYZI` types with generic template `typename PointT` type;

## 4. Future Work

* ⬜️ ;
* ⬜️ .

## Credits

This assignment was prepared by Aaron Brown and Michael Maile of Mercedes-Benz Research & Development of North America (MBRDNA), 2021 (link [here](https://learn.udacity.com/nanodegrees/nd313/)).


References
* [1] Paigwar, A. Erkent, Ö., Sierra-Gonzalez, D. Laugier, C. GndNet: Fast Ground Plane Estimation and Point Cloud Segmentation for Autonomous Vehicles. IEEE/RSJ International Conference on Intelligent Robots and Systems (IROS). 2020, pp. 2150-2156.
* [2] Gomes, T. Matias, Campos, D. Cunha, A. Roriz, R. A Survey on Ground Segmentation Methods for Automotive LiDAR Sensors. Sensors. 2023, 23(2):601. [doi:10.3390/s23020601](https://doi.org/10.3390/s23020601).
* [3] Thrun, S. Exploring artificial intelligence in the new millenium, chapter robotic mapping: a survey. Morgan Kaufmann. 2002, pp.1-35. 2002. [doi:10.5555/779343.779345](https://dl.acm.org/doi/10.5555/779343.779345).
* [4] Hu X., Florez S.A.R., Gepperth A. A Multi-Modal System for Road Detection and Segmentation. IEEE Intelligent Vehicles Symposium. 2014, pp.1365-1370. [HAL:hal-01023615](https://hal.science/hal-01023615).
* [5] Gieger, A. Lenz, P. Urtasun, R. Are we ready for autonomous driving? The KITTI vision benchmark suite. IEEE Conference on Computer Vision and Pattern Recognition. 2012. [doi:10.1109/CVPR.2012.6248074](https://ieeexplore.ieee.org/abstract/document/6248074).


Helpful resources:
* 